In [1]:
# (C) Copyright 1996- ECMWF.
#
# This software is licensed under the terms of the Apache Licence Version 2.0
# which can be obtained at http://www.apache.org/licenses/LICENSE-2.0.
# In applying this licence, ECMWF does not waive the privileges and immunities
# granted to it by virtue of its status as an intergovernmental organisation
# nor does it submit to any jurisdiction.

In [2]:
from pathlib import Path
import multiprocessing
import tqdm

import numpy as np
import pandas as pd
import xarray as xr

## User inputs

In [3]:
bootstr = 1000 # Number of bootstraps for assessing the statistical significance of the results

# get the index values of the 5th, 95th and median number, when data are ordered (for the bootstraping)
l_min = int(bootstr*5/100)
l_max = int(bootstr*95/100)-1
l_med = int(bootstr/2)

In [4]:
forecasts_loc = ''
out_dir = ''

season_studied = 'Winter' # All, Winter or Summer!
observational_dataset = 'EOBS' # ERA5 or EOBS
rolling_days = 3 # what temporal resolution to analyse? in number of days; integer>=1

## Data reading

In [5]:
aux_name = f'{observational_dataset}_{season_studied}_Days{rolling_days}'
results_used = f'{aux_name}/Statistics_{aux_name}_Bootstraps_FullAreas_Data/'

all_files = !ls $out_dir$results_used
areas_used = sorted(list(set([i.split('_')[0] for i in all_files])))
areas_used = [i for i in areas_used if i != 'Med'] # Med is only used for forecasting (this script not previous)

del(aux_name)

In [6]:
precip_data = {i_area: {} for i_area in areas_used}
for i_area in areas_used:
    
    file_name = f'{out_dir}../Data/Forecasts/{i_area}_precip.nc'
    file_data = xr.open_dataarray(file_name)
    file_data = file_data.rolling(step=rolling_days).sum()
    precip_data[i_area]['Forecasts'] = file_data.dropna('step')
    file_name = f'{out_dir}{results_used}{i_area}_PrecipERA5_Timeseries.nc'
    file_data = xr.open_dataarray(file_name)
    precip_data[i_area]['ERA5'] = file_data.dropna('time')
    file_name = f'{out_dir}{results_used}{i_area}_ReferenceConnectionsERA5_ConnTemp.nc'
    file_data = xr.open_dataarray(file_name)
    precip_data[i_area]['Climatology'] = file_data

del(i_area, file_data, file_name)

In [7]:
precip_data['Calabria']['Forecasts']

<xarray.DataArray 'tp' (WarnArea: 9, time: 2200, number: 11, step: 43)>
array([[[[ 3.74399417e+00,  5.76460089e+00,  7.40983513e+00, ...,
           5.69053586e-01,  5.38435013e-02,  1.94207883e-03],
         [ 1.30270305e-01,  6.61751832e-01,  1.41472212e+00, ...,
           6.16540152e-02,  1.14794567e-02,  1.44189218e-02],
         [ 3.73169589e+00,  6.07153485e+00,  6.74769837e+00, ...,
          -1.60982339e-15, -1.60982339e-15, -1.60982339e-15],
         ...,
         [ 2.86092233e+00,  9.37524947e+00,  1.04618859e+01, ...,
           3.56272035e-02,  2.93970504e-01,  2.93970504e-01],
         [ 1.29394717e+00,  2.41009030e+00,  2.67796721e+00, ...,
           1.25432342e-01,  7.02753411e-02,  1.34293528e-01],
         [ 1.18035631e+00,  2.57293823e+00,  3.70263151e+00, ...,
           2.56350835e-04, -8.50881865e-16, -8.50881865e-16]],

        [[ 1.34650258e+00,  6.84884693e+00,  6.83713726e+00, ...,
           2.47221049e+00,  6.48944698e-01,  1.02075350e-02],
         [ 6.49857657e-02,  4.68697719e+00,  4.77961740e+00, ...,
           5.62966730e+00,  1.02816070e+00,  8.04742928e-01],
         [ 3.34328689e+00,  1.19395138e+01,  1.19317788e+01, ...,
           1.31131655e-01,  1.10392744e-01,  5.22540886e-02],
...
         [ 2.47767036e+00,  4.61824747e-01,  3.36233687e-01, ...,
           2.65145218e+00,  1.45556573e+00,  7.51599996e-01],
         [ 1.59740590e+00,  8.44752106e-01,  3.48547158e-01, ...,
           6.72880924e+00,  5.26867780e+00,  5.02229940e+00],
         [ 2.10394616e+00,  9.01718684e-01,  3.18652293e-01, ...,
           3.39801108e-01,  1.35295238e-01,  1.50570323e+00]],

        [[ 4.20026953e-02,  2.09284421e-02,  9.43401043e-02, ...,
           3.04816993e-02,  1.03341997e-02,  1.92495304e-02],
         [ 1.63799547e-01,  1.30669655e-01,  3.22823504e-01, ...,
           2.72765253e-01,  2.75362860e-01,  2.02018652e-01],
         [ 9.89919256e-03,  5.35644835e-03,  2.46203886e-02, ...,
           1.34065447e-02,  5.34472286e-03,  9.16875176e-04],
         ...,
         [ 5.20705602e-01,  3.03553703e-01,  3.57888778e-01, ...,
           1.00492881e-01,  1.00547566e-01,  7.76621279e-02],
         [ 8.92434398e-02,  1.26769958e-01,  2.96608685e-01, ...,
           8.23289271e-04,  2.58135115e-02,  9.86027903e-02],
         [ 4.48022753e-02,  1.70319415e-02,  7.35511428e-02, ...,
           1.61664152e-02,  3.72617832e-02,  3.58441363e-02]]]])
Coordinates:
  * WarnArea  (WarnArea) int64 0 1 2 3 4 5 6 7 8
  * time      (time) datetime64[ns] 1999-06-13 2000-06-13 ... 2019-06-29
  * number    (number) int64 0 1 2 3 4 5 6 7 8 9 10
  * step      (step) timedelta64[ns] 2 days 3 days 4 days ... 43 days 44 days

In [7]:
def normalize(data):
    
    return (data-data.min('time'))/(data.max('time')-data.min('time'))

In [8]:
wvf_data = {i_area: {} for i_area in areas_used}
for i_area in areas_used:
    
    file_name = f'{out_dir}{results_used}{i_area}_WvfERA5_Timeseries.nc'
    file_data = xr.open_dataarray(file_name)
    wvf_dircs = file_data.wvf_direction.values
    file_name = f'{out_dir}{results_used}{i_area}_RhERA5_Timeseries.nc'
    file_data2 = xr.open_dataarray(file_name).rename({'pressure_level': 'wvf_direction'})
    file_data = xr.concat([file_data, file_data2], dim='wvf_direction')
    
    wvf_data[i_area]['ERA5'] = file_data.dropna('time')
    
    file_name = f'{out_dir}../Data/Forecasts/{i_area}_wvf.nc'
    file_data = xr.open_dataarray(file_name)
    file_data = file_data.rolling(step=rolling_days).mean()
    total_area = np.sqrt(file_data.sel(wvf_direction='NorthW')**2+file_data.sel(wvf_direction='EastW')**2)
    total_area = total_area.assign_coords({'wvf_direction': 'Total'})
    file_data = [file_data.sel(wvf_direction=['NorthW', 'SouthW', 'EastW', 'WestW']), total_area]
    file_data = xr.concat(file_data, dim='wvf_direction')
    
    file_name = f'{out_dir}../Data/Forecasts/{i_area}_relhum.nc'
    file_data2 = xr.open_dataarray(file_name).rename({'pressure_level': 'wvf_direction'})
    file_data2 = file_data2.assign_coords({'wvf_direction': 
                                           [f'RH{int(i)}' for i in file_data2.wvf_direction.values]})
    file_data2 = file_data2.sel(wvf_direction=wvf_data[i_area]['ERA5'].wvf_direction.values[-1])
    file_data2 = file_data2.rolling(step=rolling_days).mean()
    file_data = xr.concat([file_data, file_data2], dim='wvf_direction')
    
    wvf_data[i_area]['Forecasts'] = file_data.dropna('step')
    
    file_name = f'{out_dir}{results_used}{i_area}_ReferenceConnectionsWVF_ConnTemp.nc'
    file_data = xr.open_dataarray(file_name)
    
    wvf_data[i_area]['Climatology'] = file_data

del(i_area, file_name, file_data, total_area, file_data2)

## Data analysis - auxiliary functions

In [9]:
def define_extremes_obs(data, percentiles):
    
    quant_data = data.quantile(np.array(percentiles)/100, interpolation='linear', dim='time') # thresh.
    quant_data = quant_data.rename({'quantile': 'percentile'}) # rename coordinate
    quant_data = quant_data.assign_coords({'percentile': percentiles}) # assign the dim values based on lags

    # boolean xarray for identifying if an event is over the threshold
    exceed_xr = [data>quant_data.sel(percentile=i_p) for i_p in percentiles] 
    exceed_xr = xr.concat(exceed_xr, dim=pd.Index(percentiles, name='percentile')) # concat. data for all perc.
   
    return exceed_xr

In [10]:
# function for getting subset of forecasted precip/wvf data and generating boolean for exceedance of extremes
def define_extremes_frcst(data, percentiles):
    
    data_stack = data.stack(all_data=['time', 'number'])    
    
    quants = data_stack.quantile(np.array(percentiles)/100, 
                                 interpolation='linear', dim='all_data', keep_attrs=True) # thresholds
    quants = quants.rename({'quantile': 'percentile'}) # rename coordinate
    quants = quants.assign_coords({'percentile': percentiles}) # assign the dim values based on percentiles

    # boolean xarray for identifying if an event is over the threshold
    exceed_xr = [data>quants.sel(percentile=i_p) for i_p in percentiles] # boolean of exceedance per percentile
    exceed_xr = xr.concat(exceed_xr, dim=pd.Index(percentiles, name='percentile')) # concat. data for all perc.
    
    return exceed_xr

In [11]:
# function for getting temporal flags based on different subsetting (used later on for climatological freqs)
def temp_flagging(valid_dates, temp_subset):
    
    valid_dates = pd.to_datetime(valid_dates)
    
    if temp_subset == 'All':
        temporal_flag = ['All']*len(valid_dates)
    elif temp_subset == 'HalfYear':
        temporal_flag = (valid_dates.month%12 + 3)//3
        temporal_flag = temporal_flag.map({1: 'WinterHalf', 2: 'SummerHalf', 3: 'SummerHalf', 4: 'WinterHalf'})
    elif temp_subset == 'Season':
        temporal_flag = (valid_dates.month%12 + 3)//3
        temporal_flag = temporal_flag.map({1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Autumn'})
    elif temp_subset == 'Month':
        temporal_flag = valid_dates.month.astype(str)
    elif temp_subset == 'DayMonth':
        temporal_flag = pd.Series([i[-4:] for i in valid_dates.strftime('%Y%m%d')])
        temporal_flag = temporal_flag.values
        
    return temporal_flag  

In [12]:
def brier_score(dates_used):
    
    observations_raw = era5_lead_all.sel(time=dates_used) # keep only common dates
    forecasts_raw = frcst_lead_all.sel(time=dates_used).drop('step') # keep only common dates
    
    # if analysis for wvf, then generate also the combined predictors
    if 'wvf_direction' in observations_raw.dims:
        rh_level = era5_freqs.Extra.values[1]

        rh_alone = observations_raw.sel(wvf_direction=rh_level)
        norm = normalize(observations_raw)
        norm1 = (norm+normalize(rh_alone).drop('wvf_direction').reset_coords(drop=True))/2
        norm1 = norm1.expand_dims({'Extra': [rh_level]})
        observations_raw = observations_raw.expand_dims({'Extra': ['Alone']})
        observations_raw = xr.concat([observations_raw, norm1], dim='Extra')
        
        
        rh_alone = forecasts_raw.sel(wvf_direction=rh_level)
        norm = normalize(forecasts_raw)
        norm1 = (norm+normalize(rh_alone).drop('wvf_direction').reset_coords(drop=True))/2
        norm1 = norm1.expand_dims({'Extra': [rh_level]})
        forecasts_raw = forecasts_raw.expand_dims({'Extra': ['Alone']})
        forecasts_raw = xr.concat([forecasts_raw, norm1], dim='Extra')   
    
    observ = define_extremes_obs(observations_raw, era5_freqs.percentile.values)
    forecasts = define_extremes_frcst(forecasts_raw, era5_freqs.percentile.values)

    # generate reference forecasts
    ref_frcst_1 = era5_freqs.sel(time=temp_flagging(observ.time.values, 'Season'))
    ref_frcst_1 = ref_frcst_1.assign_coords({'time': observ.time.values})

    ref_frcst_2 = era5_freqs.sel(time=temp_flagging(observ.time.values, 'DayMonth'))
    ref_frcst_2 = ref_frcst_2.assign_coords({'time': observ.time.values})

    # get brier score
    brier_ref1 = ((observ-ref_frcst_1)**2).mean('time') # calculate brier score of ref forecast 1
    brier_ref2 = ((observ-ref_frcst_2)**2).mean('time') # calculate brier score of ref forecast 2
    brier_frcst = ((observ-forecasts.mean('number'))**2).mean('time') # calculate brier score of forecasts
    n_members = len(forecasts.number.values)
    correct_counts = (forecasts==observ).sum('number')
    fair_adjustment = correct_counts*(n_members-correct_counts)/n_members**2/(n_members-1)
    fair_brier_frcst = brier_frcst - fair_adjustment.mean('time')
    brier_all = [fair_brier_frcst, brier_frcst, brier_ref1, brier_ref2]
    brier_all = xr.concat(brier_all, dim=pd.Index(['frcst_fair', 'frcst', 'ref1', 'ref2'], name='forecast_type'))
    brier_all.name = 'BS'
    
    # get brier skill score
    brier_ref_min = brier_all.sel(forecast_type=['ref1', 'ref2']).min('forecast_type')
    brier_skill_all = 1-brier_all/brier_ref_min
    brier_skill_all.name = 'BSS'
    
    brier_all = xr.merge([brier_all, brier_skill_all]) # combine brier score and brier skill score
    
    return brier_all    

In [13]:
def brier_score_bootstrap(lead_used):
    
    global era5_lead_all, frcst_lead_all, step_used # global variables for using on functions
    step_used = lead_used
    era5_lead_all = era5
        
    frcst_lead_all = frcst.sel(step=lead_used) # subset lead day of interest for forecasts
    frcst_lead_all = frcst_lead_all.assign_coords({'time': frcst_lead_all.time+np.timedelta64(step_used, 'D')})

    common_dates = set(frcst_lead_all.time.values) & set(era5_lead_all.time.values) # get common dates
    common_dates = sorted(common_dates) # convert to sorted listed
    
    np.random.seed(10)
    bootstrap_indices = np.random.choice(common_dates, len(common_dates)*bootstr) # generate all bootstraps
    bootstrap_indices = np.array_split(bootstrap_indices, bootstr) # split into the number of subsets (samples)
    bootstrap_indices = np.array(bootstrap_indices)
    bootstrap_indices = np.insert(bootstrap_indices, 0, np.array(common_dates), axis=0) # add actual at 1st place
    
    brier_score_all_bootstraps = [brier_score(i_dates) for i_dates in bootstrap_indices]
    bootstrap_dim = pd.Index(range(bootstr+1), name='bootstrap')
    brier_score_all_bootstraps = xr.concat(brier_score_all_bootstraps, dim=bootstrap_dim)
    
    # process bootstraps for getting the results from the Q5, Q95, Median, and Actual bootstraps
    data_final = brier_score_all_bootstraps.to_array().rename({'variable': 'Indicator'})
    data_final = data_final.transpose(..., 'bootstrap')
    
    # get percent of bootstraps that BSS is positive (for checking significance of results)
    sign = (data_final.sel(Indicator='BSS')>0).sum('bootstrap')/(bootstr+1)
    sign = sign.expand_dims('Indicator').to_dataset('Indicator')
    sign = sign.rename({'BSS': 'BSS_Sign'})
    
    # get the quantiles of interest based on bootstraps
    data_quant = np.sort(data_final.isel(bootstrap=data_final.bootstrap>0), axis=-1)[..., [l_min, l_max]]
    data_quant = data_final.sel(bootstrap=range(2))*0+data_quant
    data_quant = data_quant.assign_coords({'bootstrap': ['Q5', 'Q95']})

    # add results from original analysis
    data_actual = data_final.isel(bootstrap=0).assign_coords({'bootstrap': 'Actual'})

    # get median value based on bootstraps and actual data (so median is actual sorted index)
    data_median = np.sort(data_final, axis=-1)[..., l_med]
    data_median = data_final.sel(bootstrap=0)*0+data_median
    data_median = data_median.assign_coords({'bootstrap': 'Q50'})

    data_final = xr.concat([data_quant, data_actual, data_median], dim='bootstrap')
    data_final = data_final.to_dataset('Indicator')
    
    data_final = xr.merge([data_final, sign])
    
    del(era5_lead_all, frcst_lead_all, step_used)
    
    return data_final

In [14]:
for i_area in areas_used:
    
    print(f'Brier Score calculations for {i_area} started.')
    for i_type, i_name in zip([wvf_data, precip_data], ['WVF', 'PrecipERA5']):
        frcst = i_type[i_area]['Forecasts']
        frcst = frcst.assign_coords({'step': (frcst.step.values/np.timedelta64(1, 'D')).astype(int)})
        steps_all = frcst.step.values
        
        era5 = i_type[i_area]['ERA5']
        if season_studied == 'Winter':
            era5 = era5.isel(time=pd.to_datetime(era5.time.values).month.isin([1,2,9,10,11,12]))
        elif season_studied == 'Summer':
            era5 = era5.isel(time=pd.to_datetime(era5.time.values).month.isin([3,4,5,6,7,8]))
        
        frcst.name = era5.name

        # get climatological frequencies (for DayMonth and Seasonal splitting)
        era5_freqs = i_type[i_area]['Climatology'].rename({'temporal': 'time'})
        # in case analysis is for winter and there is no climatoly for 29th Feb, give same values as 28th Feb
        if '0229' not in era5_freqs.time.values and season_studied!='Summer':
            era5_freqs = [era5_freqs, era5_freqs.sel(time='0228').assign_coords({'time': '0229'})]
            era5_freqs = xr.concat(era5_freqs, dim='time')
            
        pool = multiprocessing.Pool() # object for multiprocessing
        bs_final = list(tqdm.tqdm(pool.imap(brier_score_bootstrap, steps_all), 
                                  total=len(steps_all), position=0, leave=True))
        pool.close()
        bs_final = xr.concat(bs_final, dim=pd.Index(steps_all, name='step'))
        del(pool)

        bs_final.to_netcdf(out_dir+results_used+i_area+f'_{i_name}_ForecastingBrier.nc')
        i_type[i_area]['Brier'] = bs_final
        del(frcst, steps_all, era5, era5_freqs, bs_final)
        print(f'{i_name} Brier Score calculations for {i_area} completed.\n')      

    del(i_type, i_name)

del(i_area)

Brier Score calculations for Calabria started.


100%|███████████████████████████████████████████| 44/44 [29:09<00:00, 39.77s/it]


WVF Brier Score calculations for Calabria completed.



100%|███████████████████████████████████████████| 43/43 [06:01<00:00,  8.40s/it]

PrecipERA5 Brier Score calculations for Calabria completed.

